In [1]:
import pandas as pd
import censusdata

In [2]:
latch_df = pd.read_csv("../data/raw/latch_2017.csv")

In [3]:
latch_df.shape

(73056, 128)

In [5]:
latch_df.columns

Index(['geocode', 'Cluster', 'urban_group', 'flag_acs_lt_moe',
       'flag_manhattan_trt', 'flag_gpqtr', 'flag_incomplete_acs', 'est_pmiles',
       'est_ptrp', 'est_vmiles',
       ...
       'vmiles_3mem_0veh', 'vmiles_3mem_1veh', 'vmiles_3mem_2veh',
       'vmiles_3mem_3veh', 'vmiles_3mem_4veh', 'vmiles_4mem_0veh',
       'vmiles_4mem_1veh', 'vmiles_4mem_2veh', 'vmiles_4mem_3veh',
       'vmiles_4mem_4veh'],
      dtype='object', length=128)

## Determine which FIP codes are in Milton

In [8]:
states = censusdata.geographies(censusdata.censusgeo([('state', '25')]), 'acs5', 2020)
states

{'Massachusetts': censusgeo((('state', '25'),))}

In [13]:
norfolk_county = censusdata.geographies(censusdata.censusgeo([('state', '25'), ('county', '021')]), 'acs5', 2010)
norfolk_county

{'Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021')))}

In [18]:
milton = censusdata.geographies(censusdata.censusgeo([('state', '25'), ('county', '021'), ('tract', '*')]), 'acs5', 2020)
milton

{'Census Tract 4025, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '402500'))),
 'Census Tract 4031, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '403100'))),
 'Census Tract 4033, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '403300'))),
 'Census Tract 4034, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '403400'))),
 'Census Tract 4035.01, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '403501'))),
 'Census Tract 4035.02, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '403502'))),
 'Census Tract 4041, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '404100'))),
 'Census Tract 4042.01, Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'), ('tract', '404201'))),
 'Censu

In [35]:
latch_df.loc[latch_df.geocode.astype(str).str.startswith("250214172")]

,geocode,Cluster,urban_group,flag_acs_lt_moe,flag_manhattan_trt,flag_gpqtr,flag_incomplete_acs,est_pmiles,est_ptrp,est_vmiles,...,vmiles_3mem_0veh,vmiles_3mem_1veh,vmiles_3mem_2veh,vmiles_3mem_3veh,vmiles_3mem_4veh,vmiles_4mem_0veh,vmiles_4mem_1veh,vmiles_4mem_2veh,vmiles_4mem_3veh,vmiles_4mem_4veh
1079,25021417200,1,1.0,0,0,0,0,39.27,7.78,23.21,...,5.83,21.37,38.99,39.11,39.4,5.95,21.49,39.12,39.24,39.53


In [29]:
milton_fips = [
  "25021416101",
  "25021416102",
  "25021416200",
  "25021416300",
  "25021416400",
  "25021417202",
]

In [30]:
# By visual inspection, I think these are the tracts
"""
tracts:
  - 4161.01
  - 4161.02
  - 4162
  - 4163
  - 4164
  - 4172.02

FIPS:
  - 25021416101
  - 25021416102
  - 25021416200
  - 25021416300
  - 25021416400
  - 25021417202
"""

'\ntracts:\n  - 4161.01\n  - 4161.02\n  - 4162\n  - 4163\n  - 4164\n  - 4172.02\n\nFIPS:\n  - 25021416101\n  - 25021416102\n  - 25021416200\n  - 25021416300\n  - 25021416400\n  - 25021417202\n'

In [41]:
census_vmt = latch_df.loc[latch_df.geocode.astype(str).isin(milton_fips), ["geocode", "est_vmiles", "hh_cnt"]].set_index("geocode")
census_vmt

,est_vmiles,hh_cnt
geocode,,
25021416200,32.51,2246
25021416300,29.40,1787
25021416400,32.62,2039
25021416101,48.53,1987
25021416102,41.30,921


In [43]:
weekdays_per_year = 52*5
annual_vmt = census_vmt["est_vmiles"]*census_vmt["hh_cnt"]*weekdays_per_year
annual_vmt

geocode
25021416200    18984539.6
25021416300    13659828.0
25021416400    17293166.8
25021416101    25071568.6
25021416102     9889698.0
dtype: float64

In [44]:
annual_vmt.sum()

84898801.0

From 2017 GHG inventory, annual miles driven within Milton

```
Passenger Car				277,922,972
Light-Duty Truck				45,162,483
Medium- and Heavy-Duty Truck	24,318,260
```

In [51]:
total_vmt_in_milton = 277922972 + 45162483 + 24318260
pct_passenger = 277922972 / total_vmt_in_milton
pct_ld_truck = 45162483 / total_vmt_in_milton
pct_mhd_truck = 24318260 / total_vmt_in_milton

print(f"According to DOT, {total_vmt_in_milton} miles were driven in Milton in 2017.")
print(f"{100*pct_passenger}% were in passenger cars, {100*pct_ld_truck}% in light duty trucks, {100*pct_mhd_truck} in medium and heavy-duty trucks")

According to DOT, 347403715 miles were driven in Milton in 2017.
80.0% were in passenger cars, 13.000000014392477% in light duty trucks, 6.9999999856075235 in medium and heavy-duty trucks


Total weekday vehicle mileage by residents of Milton in 2017 was roughly 25% of the vehicle mileage within Milton during that year.  Note that the former is not a subset of the latter, as it would include miles driven outside Milton limits by Milton residents.

Nonetheless, this suggests that if we think in terms of the carbon footprint of Milton residents' activities of daily living, home heating and electricity are far and away the largest source of emissions.

In [52]:
annual_vmt.sum() / total_vmt_in_milton

0.24438080922652194